#Resources

* https://www.youtube.com/watch?v=ZzgUqFtxgXI
* https://www.youtube.com/watch?v=7aBRk_JP-qY
* https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/3/document-splitting

#Introduction

Title: Exam MCQ Generator

Game Plan:

*   Divide the document into multiple chuncks and then select chunks at random, pass them to Langchain and ask to make an MCQ out of it

#Installs

In [112]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

In [113]:
!pip install -q pydantic

#Imports

In [114]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [115]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
wizard_in_oz_text_path = '/content/drive/MyDrive/datasets/llm-from-scratch/wizard_in_oz.txt'

In [139]:
def read_text_file(path):
  with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

  return text

In [140]:
text = read_text_file(wizard_in_oz_text_path)

In [141]:
print(text[:100])

﻿Title: Dorothy and the Wizard in Oz


Author: L. Frank Baum

Illustrator: John R. Neill

Release da


#Env

In [119]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [120]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [121]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [122]:
from pydantic import BaseModel, Field

class MCQModel(BaseModel):
  question: str = Field(description="This is the question text")
  options: list[str] = Field(description="This is a list of multiple choices or options avalible")
  correct_option: str = Field(description="This is the correct choice or option")

In [123]:
from langchain.output_parsers import PydanticOutputParser

pydantic_parser = PydanticOutputParser(pydantic_object=MCQModel)

format_instructions = pydantic_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"question": {"title": "Question", "description": "This is the question text", "type": "string"}, "options": {"title": "Options", "description": "This is a list of multiple choices or options avalible", "type": "array", "items": {"type": "string"}}, "correct_option": {"title": "Correct Option", "description": "This is the correct choice or option", "type": "string"}}, "required": ["question", "options", "correct_option"]}
```


In [124]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

In [128]:
prompt = (
    "Text:"
    "\n{text}"
    "\nGenerate an Multiple Choice Question from this text. Also return the correct option."
    "\nDo not use any outside information"
    "\n{format_instructions}"
)

In [129]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='Text:\n{text}\nGenerate an Multiple Choice Question from this text. Also return the correct option.\nDo not use any outside information\n{format_instructions}'))])

In [130]:
llm_chain = LLMChain(llm=langchain_llm, prompt=prompt_template, output_key="result")

In [181]:
def get_docs(text, chunk_size=500, chunk_overlap=0):
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
      is_separator_regex=False,
  )

  docs = text_splitter.create_documents([text])

  return docs

In [178]:
from langchain.callbacks import get_openai_callback as get_langchain_openai_callback
import random

def get_mcq(docs):
  doc_index = random.randint(0, len(docs)-1)

  with get_langchain_openai_callback() as cb_langchain:
    response = llm_chain.invoke({"text": docs[doc_index], "format_instructions": format_instructions})

  return {'doc_index': doc_index, 'mcq_parsed':  MCQModel.parse_raw(response['result']), 'cb_langchain': cb_langchain}

In [197]:
def cb_to_dict(info_string):
    # Split the string into lines
    lines = str(info_string).strip().split('\n')

    # Initialize an empty dictionary
    info_dict = {}

    # Iterate through each line and extract key-value pairs
    for line in lines:
        key, value = map(str.strip, line.split(':', 1))
        # Check if the value is numeric, convert it to an int or float accordingly
        if value.replace('.', '', 1).isdigit():
            value = float(value) if '.' in value else int(value)
        info_dict[key] = value

    info_dict['Total Cost (USD)'] = float(info_dict['Total Cost (USD)'].replace('$', ''))

    return info_dict

In [185]:
def sum_dicts(dict_list):
    if not dict_list:
        return {}

    result_dict = {}

    # Find the intersection of keys in all dictionaries
    common_keys = set.intersection(*(set(d.keys()) for d in dict_list))

    for key in common_keys:
        result_dict[key] = sum(d[key] for d in dict_list)

    return result_dict

In [199]:
def get_exam(text, limit=5):

  docs = get_docs(text)

  mcqs = [get_mcq(docs) for i in range(limit)]

  return {'mcqs': mcqs, 'sum_cost': sum_dicts([cb_to_dict(i['cb_langchain']) for i in mcqs])}

#Testing

In [174]:
test_docs = get_docs(text=text, chunk_size=1000, chunk_overlap=200)

In [179]:
response = get_mcq(docs=test_docs)

In [180]:
print(response['mcq_parsed'])

print('\n')
print(f"doc_index = {response['doc_index']}")
print('\ncosting:\n')
print(f"{response['cb_langchain']}")

question='What did the Wizard call himself when he joined the circus?' options=['A. Wizard', 'B. Oz', 'C. Superior creature', 'D. Ventriloquist'] correct_option='A. Wizard'


doc_index = 217

costing:

Tokens Used: 558
	Prompt Tokens: 506
	Completion Tokens: 52
Successful Requests: 1
Total Cost (USD): $0.000863


#Pydantic Demo

In [135]:
from pydantic import BaseModel, EmailStr, validator

class User(BaseModel):
  name: str
  # email: EmailStr
  email: str
  account_id: int

  @validator("account_id")
  def validate_account_id(cls, value):
    if value<=0:
      raise ValueError(f'Account ID cannot be negative: {value}')

    return value

user = User(name='jack', email='a@b.com', account_id=1234)

print(user)
print(user.json())
print(user.dict())
print(user.parse_raw(user.json()))

name='jack' email='a@b.com' account_id=1234
{"name": "jack", "email": "a@b.com", "account_id": 1234}
{'name': 'jack', 'email': 'a@b.com', 'account_id': 1234}
name='jack' email='a@b.com' account_id=1234


#Working Demo

In [226]:
def display_exam(exam):

  for i in exam['mcqs']:
    print(f"doc_index: {i['doc_index']}\n")
    print(f"question: {i['mcq_parsed'].question}\n")

    for index, j in enumerate(i['mcq_parsed'].options):
      print(f"{index+1}. {j}")

    print(f"\ncorrect: {i['mcq_parsed'].correct_option}\n\n")

  print(exam['sum_cost'])

In [206]:
texts_path = '/content/drive/MyDrive/datasets/exam_mcq_generator/texts'

In [207]:
text_path_01 = os.path.join(texts_path, '01.txt')
text_path_02 = os.path.join(texts_path, '02.txt')
text_path_03 = os.path.join(texts_path, '03.txt')
text_path_04 = os.path.join(texts_path, '04.txt')
text_path_05 = os.path.join(texts_path, '05.txt')

In [228]:
text_01 = read_text_file(text_path_01)

display(text_01)

"In the small coastal village of Eldridge Haven, where the briny scent of the ocean mingled with the sweet fragrance of blooming flowers, lived a young woman named Seraphina. Her days were filled with the rhythmic ebb and flow of tide, the gentle whispers of the wind through ancient willows, and the comforting routine of life in a close-knit community. Yet, beneath the surface of tranquil routine lurked an undercurrent of restlessness that stirred within Seraphina's soul, beckoning her toward the unknown.\n\nOne fateful evening, as the sun dipped below the horizon, casting a warm palette of pinks and purples across the sky, Seraphina took a solitary stroll along the shore. The rhythmic crashing of the waves seemed to sync with the beating of her heart, echoing the unspoken desire for adventure that tugged at the edges of her consciousness. Little did she know that this serene walk would unravel the fabric of her familiar existence and set her on a path of unforeseen destiny.\n\nIn the 

In [229]:
display(text_01[:500])

"In the small coastal village of Eldridge Haven, where the briny scent of the ocean mingled with the sweet fragrance of blooming flowers, lived a young woman named Seraphina. Her days were filled with the rhythmic ebb and flow of tide, the gentle whispers of the wind through ancient willows, and the comforting routine of life in a close-knit community. Yet, beneath the surface of tranquil routine lurked an undercurrent of restlessness that stirred within Seraphina's soul, beckoning her toward the"

In [230]:
exam_01 = get_exam(text=text_01, limit=5)

In [231]:
display_exam(exam_01)

doc_index: 5

question: What did Seraphina become after her journey?

1. A curious villager
2. A guardian of realms
3. A watchful eye
4. A delicate tapestry

correct: A guardian of realms


doc_index: 2

question: What did Seraphina find in the sand?

1. An ancient artifact
2. A treasure chest
3. A seashell
4. A mermaid

correct: An ancient artifact


doc_index: 6

question: What does Eldridge Haven stand as a testament to?

1. The enduring power of curiosity
2. The enduring power of courage
3. The uncharted territories within every soul
4. All of the above

correct: All of the above


doc_index: 4

question: What did Seraphina discover in the archipelago?

1. Lost civilizations
2. Enigmatic beings
3. Mythical creatures
4. A realm suspended between reality and dreams

correct: A realm suspended between reality and dreams


doc_index: 5

question: What did Seraphina become after her journey?

1. A curious villager
2. A guardian of realms
3. A watchful eye
4. A young woman

correct: A gu

In [232]:
text_02 = read_text_file(text_path_02)

display(text_02)

'In the not-so-distant future, a revolutionary technological marvel known as SynthEra emerged from the laboratories of Quantum Dynamics Innovations, forever altering the landscape of human experience. SynthEra, a groundbreaking neural interface, represented the pinnacle of advancements in neurotechnology, seamlessly blending the digital and biological realms.\n\nAt its core, SynthEra was designed to establish a direct and symbiotic connection between the human brain and the vast expanse of digital information. Consisting of a sleek neuro-interface headset adorned with an intricate web of sensors, SynthEra employed advanced neurostimulation algorithms to interpret neural signals, transforming thoughts into actionable commands. The device seamlessly integrated with the user\'s cognitive functions, creating an immersive and intuitive interface for navigating the digital world.\n\nThe possibilities unlocked by SynthEra were nothing short of awe-inspiring. Users could interact with augmente

In [233]:
exam_02 = get_exam(text=text_02, limit=5)

In [234]:
display_exam(exam_02)

doc_index: 6

question: What did SynthEra bring forth?

1. A future where the mind's potential was harnessed
2. A future where the physical world's limitations were transcended
3. An era where biology and technology converged
4. All of the above

correct: All of the above


doc_index: 6

question: What did SynthEra bring forth?

1. A future where the mind's potential was harnessed
2. A future where the physical world's limitations were transcended
3. An era where biology and technology converged
4. All of the above

correct: All of the above


doc_index: 6

question: What did SynthEra bring forth?

1. A future where the mind's potential was harnessed
2. A future where biology and technology converged
3. A future where the physical world's limitations were transcended

correct: A future where biology and technology converged


doc_index: 2

question: What could users do with SynthEra?

1. Access augmented reality overlays
2. Manipulate the virtual environment
3. Retrieve information ins

In [235]:
text_03 = read_text_file(text_path_03)

display(text_03)

"Deep within the esoteric realm of abstract mathematics, a conundrum known as the Collatz conjecture captivated the minds of scholars and amateurs alike. Born in the mid-20th century from the intellect of German mathematician Lothar Collatz, this seemingly straightforward problem birthed a puzzle that has defied resolution for decades, its elusiveness challenging the very core of mathematical understanding.\n\nAt its heart, the Collatz conjecture revolves around the interplay of natural numbers. The rules are deceptively simple: take any positive integer 'n,' if it is even, divide it by 2; if it is odd, triple it and add 1. Repeat this process iteratively, and the conjecture posits that, regardless of the starting value, the sequence will eventually reach the elusive cycle of 4, 2, 1, and from thereon endlessly loop in this enigmatic dance.\n\nMathematicians, drawn to the allure of simplicity masking complexity, embarked on a journey to unlock the secrets embedded in the Collatz conjec

In [236]:
exam_03 = get_exam(text=text_03, limit=5)

In [237]:
display_exam(exam_03)

doc_index: 4

question: What did the collaborative efforts in the mathematical community lead to?

1. New research papers
2. Computational approaches
3. Philosophical musings
4. Camaraderie and shared intrigue

correct: Camaraderie and shared intrigue


doc_index: 4

question: What blossomed as the mathematical community grappled with the Collatz conjecture?

1. Sense of camaraderie
2. Lack of interest
3. Confusion
4. Disagreement

correct: Sense of camaraderie


doc_index: 6

question: What does the resolution of the Collatz conjecture promise?

1. A solution to a mathematical puzzle
2. A deeper understanding of the beauty of numbers
3. Both A solution to a mathematical puzzle and A deeper understanding of the beauty of numbers
4. None of the above

correct: Both A solution to a mathematical puzzle and A deeper understanding of the beauty of numbers


doc_index: 0

question: What is the name of the conundrum in abstract mathematics that has defied resolution for decades?

1. Collatz c

In [238]:
text_04 = read_text_file(text_path_04)

display(text_04)

"In the vast expanse of human linguistic articulation, poetry emerges as an enigmatic manifestation, a convolution of lexical cadence that transcends the conventional confines of mundane discourse. Within the nebulous realms of metaphorical articulation, poets, as lexical alchemists, wield a contrivance of language that begets an ethereal symphony, wherein the elusive shades of human sentiment meander.\n\nThe nuanced choreography of syllabic configurations and rhythmic resonances imbues poets with the capacity to metamorphose prosaic verbiage into an intricate mosaic of expressive profundity. In its essence, a meticulously wrought poem becomes a tapestry of allegorical impressions, each epithet and turn of phrase akin to a brushstroke upon the canvas of cognition.\n\nThe potency of poetry resides not solely within its articulated lexicon but rather within the interstices of verbal modulations, the intervals of pregnant reticence that implore the discerning reader to tarry and ruminate.

In [239]:
exam_04 = get_exam(text=text_04, limit=5)

In [240]:
display_exam(exam_04)

doc_index: 0

question: What is poetry?

1. A form of human linguistic articulation
2. A manifestation of lexical cadence
3. A convolution of mundane discourse
4. A contrivance of ethereal symphony

correct: A form of human linguistic articulation


doc_index: 7

question: What does poetry stand as in the expansive canvas of human enunciation?

1. A. An incandescent filament
2. B. A connection betwixt disparate hearts and intellects
3. C. A testament to the indomitable tenacity of language
4. D. All of the above

correct: D. All of the above


doc_index: 6

question: What does poetry serve as?

1. A sanctuary for contemplation
2. A tool for distilling meaning
3. An escape from daily life
4. All of the above

correct: All of the above


doc_index: 6

question: What does poetry serve as?

1. A sanctuary for introspection and contemplation
2. A tool for distilling meaning from a cacophonous milieu
3. A refuge for aesthetic splendors
4. All of the above

correct: All of the above


doc_ind

In [241]:
text_05 = read_text_file(text_path_05)

display(text_05)

"The history of Large Language Models (LLMs) is a fascinating journey that spans the intersection of artificial intelligence, natural language processing, and machine learning. The roots of LLMs can be traced back to the early endeavors in computational linguistics, where researchers sought to develop systems capable of understanding and generating human-like language.\n\nThe seeds of LLMs were sown in the mid-20th century, with early attempts at computer-based language translation. The Georgetown-IBM experiment in 1954 marked a pioneering effort, utilizing an early computer to translate Russian sentences into English. However, these early endeavors were constrained by the limited computational power of the time, and the results fell short of achieving true linguistic nuance.\n\nAs computational capabilities burgeoned in the latter half of the 20th century, researchers delved into more sophisticated approaches to natural language processing. Rule-based systems emerged, relying on prede

In [242]:
exam_05 = get_exam(text=text_05, limit=5)

In [243]:
display_exam(exam_05)

doc_index: 8

question: What is the focus of ongoing research on LLMs?

1. Refining the models
2. Addressing ethical concerns
3. Exploring novel applications
4. All of the above

correct: All of the above


doc_index: 7

question: What are some concerns associated with the evolution of LLMs?

1. Bias, accountability, and environmental impact
2. Scale, complexity, and societal impact
3. Responsibility, AI practices, and development
4. Training processes, researchers, and challenges

correct: Bias, accountability, and environmental impact


doc_index: 8

question: What does ongoing research on LLMs focus on?

1. Refining these models
2. Addressing ethical concerns
3. Exploring novel applications
4. All of the above

correct: All of the above


doc_index: 0

question: The history of Large Language Models (LLMs) is a fascinating journey that spans the intersection of which fields?

1. Artificial intelligence
2. Natural language processing
3. Machine learning

correct: All of the above


do

#Solutions

**Repeating Questions: Do not repeat the random doc number**